In [106]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision
import os
import numpy as np


transform = transforms.Compose([#transforms.Resize((256,256)),  
                                transforms.Grayscale(),		# the code transforms.Graysclae() is for changing the size [3,100,100] to [1, 100, 100] (notice : [channel, height, width] )
                                transforms.ToTensor(),])


#train_data_path = 'relative path of training data set'
train_data_path = './horse-or-human/train'
trainset = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
# if shuffle=True, the data reshuffled at every epoch 
trainloader = torch.utils.data.DataLoader(trainset, batch_size=3, shuffle=False, num_workers=1)  


validation_data_path = './horse-or-human/validation'
valset = torchvision.datasets.ImageFolder(root=validation_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
valloader = torch.utils.data.DataLoader(valset, batch_size=3, shuffle=False, num_workers=1)  

NUM_EPOCH = 10



coef = np.zeros((1,10001), dtype = float)
lrnRate = 0.01


for epoch in range(NUM_EPOCH):
    # load training images of the batch size for every iteration
    
    
    sumDCoef = np.zeros((10001,1), dtype = float)
    for i, data in enumerate(trainloader):

        # inputs is the image
        # labels is the class of the image
        inputs, labels = data
        

        # if you don't change the image size, it will be [batch_size, 1, 100, 100]
        batch_size = inputs.shape[0]


        # if labels is horse it returns tensor[0,0,0] else it returns tensor[1,1,1]

        
        # change inputs to matrix 10000*batch_size
        
        for bat_idx in range(batch_size):
            
            targMat = inputs[bat_idx][0]
            colVec = np.reshape(targMat, (np.product(targMat.shape), 1), 'F')
            
            if(bat_idx == 0):
                batMat = colVec
            else:
                batMat = np.concatenate((batMat, colVec), axis = 1)
        
        ones = np.ones((1, batch_size), dtype = int)
        batMat = np.concatenate((batMat, ones))
        
        
        # Start Regression Calculation
        z = np.dot(coef, batMat)
        a = 1/(1 + np.exp(-z))
        dz = np.subtract(a, labels)
        #change ndarray to tensor
        batMat = torch.from_numpy(batMat)
        dCoef = dz * batMat
        sumHelper = np.ones((batch_size,1), dtype = int)
        sumDCoef += np.dot(dCoef, sumHelper)


    sumDCoef /= len(trainloader.dataset)
    coef = coef.T
    

    coef -= lrnRate * sumDCoef
    print(coef)
    coef = coef.T


        
                

                
            
                
        
        





    # load validation images of the batch size for every iteration
    for i, data in enumerate(valloader):
        
        # inputs is the image
        # labels is the class of the image
        inputs, labels = data

        # if you don't change the image size, it will be [batch_size, 1, 100, 100]
        #print(inputs.shape)

        # if labels is horse it returns tensor[0,0,0] else it returns tensor[1,1,1]
        #print(labels)    
        
    

[[0.0001624 ]
 [0.00014438]
 [0.0001275 ]
 ...
 [0.00021496]
 [0.00021487]
 [0.00013145]]
[[-0.00132658]
 [-0.00136319]
 [-0.00139753]
 ...
 [-0.00133803]
 [-0.00133458]
 [-0.00250029]]
[[0.00158416]
 [0.0015302 ]
 [0.00147491]
 ...
 [0.00171714]
 [0.00171676]
 [0.00247992]]
[[-0.00104237]
 [-0.00111475]
 [-0.00118169]
 ...
 [-0.00094602]
 [-0.00094302]
 [-0.00235847]]
[[0.00169065]
 [0.00160138]
 [0.00151577]
 ...
 [0.00192615]
 [0.0019247 ]
 [0.00218767]]
[[-0.00094067]
 [-0.00104831]
 [-0.00114537]
 ...
 [-0.00074111]
 [-0.00073918]
 [-0.00266556]]
[[0.00124999]
 [0.00112629]
 [0.00101302]
 ...
 [0.001528  ]
 [0.00152527]
 [0.00081443]]
[[-0.0013757 ]
 [-0.00151782]
 [-0.00164274]
 ...
 [-0.00113531]
 [-0.00113466]
 [-0.00402388]]
[[0.00132078]
 [0.00116181]
 [0.00101838]
 ...
 [0.00167483]
 [0.00167082]
 [0.00042363]]
[[-0.0013093 ]
 [-0.00148665]
 [-0.00164156]
 ...
 [-0.00099189]
 [-0.00099254]
 [-0.00442732]]


In [56]:
a = np.array([1,2,3])[np.newaxis]
b = np.array([4,5,6])[np.newaxis]
c = np.array([7,8,9])[np.newaxis]
a = a.T
b = b.T
c = c.T

x = a



print(x)
x = np.concatenate((x,b),axis=1)
print(x)
x = np.concatenate((x,c),axis=1)
print(x)
ones = np.ones((1,3), dtype=int)
print(ones)
x = np.concatenate((x,ones))
print(x)

'''
x = np.column_stack((a,b))
print(x)

x = np.column_stack((x,c))
print(c)
'''







[[1]
 [2]
 [3]]
[[1 4]
 [2 5]
 [3 6]]
[[1 4 7]
 [2 5 8]
 [3 6 9]]
[[1 1 1]]
[[1 4 7]
 [2 5 8]
 [3 6 9]
 [1 1 1]]


'\nx = np.column_stack((a,b))\nprint(x)\n\nx = np.column_stack((x,c))\nprint(c)\n'

In [25]:
t = np.array([[1,2,3,4],
             [4,5,6,6],
             [7,8,9,10]])

print(t)
print(t.shape)
print(np.product(t.shape))
b = np.reshape(t, (np.product(t.shape), 1), 'F')
print(b)



'''
t = t.T


b = np.reshape(t, (1 ,np.product(t.shape))).T
print(b)
'''

[[ 1  2  3  4]
 [ 4  5  6  6]
 [ 7  8  9 10]]
(3, 4)
12
[[ 1]
 [ 4]
 [ 7]
 [ 2]
 [ 5]
 [ 8]
 [ 3]
 [ 6]
 [ 9]
 [ 4]
 [ 6]
 [10]]


'\nt = t.T\n\n\nb = np.reshape(t, (1 ,np.product(t.shape))).T\nprint(b)\n'

In [51]:
a = np.array([[1,2,3]
             ,[4,5,6]])
print(a)

a = np.matmul(3, a)
print(a)

[[1 2 3]
 [4 5 6]]


ValueError: matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [65]:
z = np.array([[3,3,3],
             [3,3,3],
             [3,3,3],
             [3,3,3]])
print(z)

a = np.array([1,2,3])

print(a)

x = a * z
print(x)


[[3 3 3]
 [3 3 3]
 [3 3 3]
 [3 3 3]]
[1 2 3]
[[3 6 9]
 [3 6 9]
 [3 6 9]
 [3 6 9]]


In [80]:
z = np.array([[1],
            [2],
            [3]])
x = np.zeros((3,1), dtype = int)
print(z)
print(x)
x = np.add(z,x)
print(x)

x = np.add(x,z)
print(x)

[[1]
 [2]
 [3]]
[[0]
 [0]
 [0]]
[[1]
 [2]
 [3]]
[[2]
 [4]
 [6]]
